In [1]:
import pickle
from hashlib import sha1
from datasketch import MinHash, MinHashLSH
from tqdm import tqdm

In [2]:
def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

In [3]:
def hashes_simple_1(x):
    if x == 'PLACEHOLDER':
        return 'PLACEHOLDER'
    else:
        m1 = MinHash(num_perm=16)
        x = x.lower()
        _ = [m1.update(token.encode('utf8')) for token in x.split()]
        return m1

In [4]:
%%time
w0 = pickle.load(open("Wiki_pickle\save.p", "rb"))
w1 = pickle.load(open("Wiki_pickle\save1.p", "rb"))
w2 = pickle.load(open("Wiki_pickle\save2.p", "rb"))
w3 = pickle.load(open("Wiki_pickle\save3.p", "rb"))
w4 = pickle.load(open("Wiki_pickle\save4.p", "rb"))
w5 = pickle.load(open("Wiki_pickle\save5.p", "rb"))

Wall time: 19min 32s


In [5]:
from bs4 import BeautifulSoup
from html import unescape
import unicodedata
import string
import unidecode
import re

def escape_html(text):
    soup = BeautifulSoup(unescape(text), 'html.parser')
    text = soup.text
    text = unicodedata.normalize('NFKC', text)
    return text

def my_replace(match):
    match = match.group()
    return unidecode.unidecode(match)

In [6]:
def replace_dashes(text):
    dashes = '-‐‑‒–—―'
    for dash in dashes:
        text = text.replace(dash, '-')
    text = text.replace('--', '-').replace('--', '-')
    return text

def replace_quotes(text):
    quotes = '»«”“„‟❝❞〝〞〟＂'
    for q in quotes:
        if isinstance(text, str):
            text = text.replace(q, '"')
    if isinstance(text, str):
        text = text.replace(',,', '"')
        
    text = text.replace("<<", '"').replace(">>", '"').replace("'' '", '"').replace(">", '').replace("<", '')
    text = text.replace("''", '"').replace("''", '"')    
    return text

In [17]:
def process_txt_new(line):
    if line == '\n' or line == '\r\n':
        return 'PLACEHOLDER'
    else:
        line = line.replace('\n', '')
        if len(line):
            line = escape_html(line).replace('  ', ' ').replace('  ', ' ')
            line = re.sub('([^А-яЁё])', my_replace, line)
            line = replace_dashes(line)
            line = replace_quotes(line)
            line = line.replace('\r', '')
            if len(line):
                line = line.replace('  ', ' ').replace('  ', ' ') 
                if len(line):
                    remove_words = ['#перенаправление', 'Категория:', 'Файл:', 'Оглавление', '|мини|', 
                                    ' = =', 'JPG|', 'JPG','слева|', 'справа|', '|справа', '|thumb|left|200px|'
                '#ИзображениеНазваниеМестоположениеВремя созданияГод внесения в список№Критерии']
                    for word in remove_words:
                        line = line.replace(word, ' ')
                    if len(line):
                        line = re.sub('=== .+? ===', ' ', line)
                        line = re.sub('={2,}', ' ', line)
                        line = re.sub('={2,}', ' ', line)
                        line = line.strip()
                        if len(line):
                            return line

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
%%time

d = 0
lsh = MinHashLSH(threshold=0.8, num_perm=16)
r = 0

with lsh.insertion_session() as session:
    with open('d\wiki1_v_01.txt', 'w', encoding='utf-8', newline='\n') as f:
        with open('d\wiki1.txt', encoding='utf-8') as f1:
            for piece in read_in_chunks_lines(f1, chunk_size=5000000):
                sents = [process_txt_new(x) for x in piece]
                sents = [line for line in sents if line is not None]
                hashes = [hashes_simple_1(x) for x in tqdm(sents)]
                
                for i in range(len(hashes)):                   
                    if sents[i] == 'PLACEHOLDER':
                        f.write('\n')  
                    else:    
                        key = hashes[i]
                        q = len(w1.query(key))
                        q1 = len(w0.query(key))
                        q2 = len(w2.query(key))
                        q3 = len(w3.query(key)) 
                        q4 = len(w4.query(key))
                        q5 = len(w5.query(key))  

                        s =  q1 + q2 + q3 + q4 + q5


                        if q < 2 and s == 0:
                            f.write(sents[i] + '\n')
                        else:
                            if not len(lsh.query(hashes[i])): 
                                r +=1 
                                session.insert('w'+str(r), hashes[i])                
                                f.write(sents[i] + '\n')
                            else:
                                d += 1
                                if d % 100000 == 0:
                                    print(f'Duplicates found --- {d:,}')

100%|██████████████████████████████████████████████████████████████████████████| 52309/52309 [00:19<00:00, 2699.99it/s]
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://femb.ru/femb/pharmacopea.php)Категория:Фармакопея" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
100%|██████████████████████████████████████████████████████████████████████████| 49492/49492 [00:16<00:00, 2926.86it/s]
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "https://web.archive.org/web/20180126100730/http://www.sergey-grigoriev.ru/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the U

100%|██████████████████████████████████████████████████████████████████████████| 49768/49768 [00:19<00:00, 2591.88it/s]
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://alt.cimedia.com/statesman/specialreports/whitman/letter.pdf." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://www.deekmagazine.com/issues/20_Brutality/pages/Brutality_Features%20What%20Charlie%20Saw.htm." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
100%|

Duplicates found --- 100,000


c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "https://web.archive.org/web/20160303184958/http://www.bibliomedia.ch/fr/autoren/Polla_Barbara/558.html3." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "https://analixforever.wordpress.com/4." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "https://analixfor

c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://anima.ucoz.ru/vstrechi/char/char_7_srv.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
100%|██████████████████████████████████████████████████████████████████████████| 56213/56213 [00:17<00:00, 3166.16it/s]
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:336: MarkupResemblesLocatorWarning: "                              ." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "https://web.archive.org/web/20

...

c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:336: MarkupResemblesLocatorWarning: "   ." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://www.ulizza.ru/novosti/yaroslavl-pat-khoroshih-novostej-za-20-ijula-2011-goda/#.VLjEltKsWWM" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "https://books.google.com/?id=aOzohrk8Mm8C&dq=Nunez,+Greco,+Burial+of+Orgaz." looks like a URL. Beautiful Soup is not an HTTP clien

c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:336: MarkupResemblesLocatorWarning: "D.." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
100%|██████████████████████████████████████████████████████████████████████████| 52443/52443 [00:18<00:00, 2820.73it/s]


Duplicates found --- 1,000,000


100%|██████████████████████████████████████████████████████████████████████████| 55154/55154 [00:19<00:00, 2805.22it/s]
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "http://www.pogodaiklimat.ru/climate/35188.htmКатегория:Нур-СултанНур-Султан" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:424: MarkupResemblesLocatorWarning: "https://doi.org/10.1079/9780851994086.0351" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning
c:\users\администратор\appdata\local\programs\python\py

Wall time: 2h 1min 49s


In [13]:
print(f'Total duplicates in file -- {d:,}')

Total duplicates in file -- 1,097,899


In [16]:
%%time
with lsh.insertion_session() as session:
    with open('d\wiki2_v_01.txt', 'w', encoding='utf-8') as f:
        with open('d\wiki2.txt', encoding='utf-8') as f1:
            for piece in read_in_chunks_lines(f1, chunk_size=6000000):
                sents = [process_txt_new(x) for x in piece]
                sents = [line for line in sents if line is not None]
                hashes = [hashes_simple_1(x) for x in tqdm(sents)]
                
                for i in range(len(hashes)):                   
                    if sents[i] == 'PLACEHOLDER':
                        f.write('\n')  
                    else:                            
                        key = hashes[i]
                        q = len(w2.query(key))
                        q1 = len(w0.query(key))
                        q2 = len(w1.query(key))
                        q3 = len(w3.query(key)) 
                        q4 = len(w4.query(key))
                        q5 = len(w5.query(key))  

                        s =  q1 + q2 + q3 + q4 + q5

                        if q < 2 and s == 0:
                            f.write(sents[i] + '\n')
                        else:
                            if not len(lsh.query(hashes[i])): 
                                r +=1 
                                session.insert('w'+str(r), hashes[i])                
                                f.write(sents[i] + '\n')
                            else:
                                d += 1
                                if d % 100000 == 0:
                                    print(f'Duplicates found --- {d:,}')

100%|██████████████████████████████████████████████████████████████████████████| 86291/86291 [00:24<00:00, 3490.34it/s]


Duplicates found --- 1,100,000


100%|██████████████████████████████████████████████████████████████████████████| 65760/65760 [00:21<00:00, 3039.28it/s]


Duplicates found --- 1,200,000


100%|██████████████████████████████████████████████████████████████████████████| 66180/66180 [00:21<00:00, 3057.49it/s]


...

Duplicates found --- 2,000,000


100%|██████████████████████████████████████████████████████████████████████████| 65666/65666 [00:23<00:00, 2849.56it/s]


Duplicates found --- 2,100,000


100%|██████████████████████████████████████████████████████████████████████████| 43243/43243 [00:15<00:00, 2872.56it/s]


Wall time: 1h 39min 27s


In [18]:
print(f'Total duplicates in file -- {d:,}')

Total duplicates in file -- 2,193,299


In [19]:
%%time
with lsh.insertion_session() as session:
    with open('d\wiki3_v_01.txt', 'w', encoding='utf-8') as f:
        with open('d\wiki3.txt', encoding='utf-8') as f1:
            for piece in read_in_chunks_lines(f1, chunk_size=5000000):
                sents = [process_txt_new(x) for x in piece]
                sents = [line for line in sents if line is not None]
                hashes = [hashes_simple_1(x) for x in tqdm(sents)]
                
                for i in range(len(hashes)):                   
                    if sents[i] == 'PLACEHOLDER':
                        f.write('\n')  
                    else:    
                        
                        key = hashes[i]
                        q = len(w3.query(key))
                        q1 = len(w0.query(key))
                        q2 = len(w1.query(key))
                        q3 = len(w2.query(key)) 
                        q4 = len(w4.query(key))
                        q5 = len(w5.query(key))  

                        s =  q1 + q2 + q3 + q4 + q5
              
                        if q < 2 and s == 0:
                            f.write(sents[i] + '\n')
                        else:
                            if not len(lsh.query(hashes[i])): 
                                r +=1 
                                session.insert('w'+str(r), hashes[i])                
                                f.write(sents[i] + '\n')
                            else:
                                d += 1
                                if d % 100000 == 0:
                                    print(f'Duplicates found --- {d:,}')

100%|██████████████████████████████████████████████████████████████████████████| 55810/55810 [00:19<00:00, 2930.36it/s]


Duplicates found --- 2,200,000


100%|██████████████████████████████████████████████████████████████████████████| 52041/52041 [00:18<00:00, 2888.57it/s]


Duplicates found --- 2,300,000


100%|██████████████████████████████████████████████████████████████████████████| 54334/54334 [00:17<00:00, 3023.92it/s]


...

Duplicates found --- 2,800,000


100%|██████████████████████████████████████████████████████████████████████████| 57878/57878 [00:18<00:00, 3204.15it/s]


Duplicates found --- 2,900,000


100%|██████████████████████████████████████████████████████████████████████████| 32005/32005 [00:10<00:00, 3185.26it/s]


Wall time: 1h 36min 48s


In [20]:
print(f'Total duplicates in file -- {d:,}')

Total duplicates in file -- 2,955,553


In [22]:
%%time
with lsh.insertion_session() as session:
    with open('d\wiki0_v_01.txt', 'w', encoding='utf-8') as f:
        with open('d\wiki0.txt', encoding='utf-8') as f1:
            for piece in read_in_chunks_lines(f1, chunk_size=6000000):
                sents = [process_txt_new(x) for x in piece]
                sents = [line for line in sents if line is not None]
                hashes = [hashes_simple_1(x) for x in tqdm(sents)]
                
                for i in range(len(hashes)):                   
                    if sents[i] == 'PLACEHOLDER':
                        f.write('\n')  
                    else:    
                        key = hashes[i]
                        q = len(w0.query(key))
                        q1 = len(w2.query(key))
                        q2 = len(w1.query(key))
                        q3 = len(w3.query(key)) 
                        q4 = len(w4.query(key))
                        q5 = len(w5.query(key))  

                        s =  q1 + q2 + q3 + q4 + q5


                        if q < 2 and s == 0:
                            f.write(sents[i] + '\n')
                        else:
                            if not len(lsh.query(hashes[i])): 
                                r +=1 
                                session.insert('w'+str(r), hashes[i])                
                                f.write(sents[i] + '\n')
                            else:
                                d += 1
                                if d % 100000 == 0:
                                    print(f'Duplicates found --- {d:,}')

100%|██████████████████████████████████████████████████████████████████████████| 57966/57966 [00:21<00:00, 2668.43it/s]


Duplicates found --- 3,000,000


100%|██████████████████████████████████████████████████████████████████████████| 55506/55506 [00:23<00:00, 2359.61it/s]


Duplicates found --- 3,100,000


100%|██████████████████████████████████████████████████████████████████████████| 54740/54740 [00:20<00:00, 2625.91it/s]


...

Duplicates found --- 4,600,000


100%|██████████████████████████████████████████████████████████████████████████| 61783/61783 [00:20<00:00, 2991.40it/s]


Duplicates found --- 4,700,000


100%|██████████████████████████████████████████████████████████████████████████| 31158/31158 [00:10<00:00, 2926.89it/s]


Wall time: 3h 17min 25s


In [23]:
print(f'Total duplicates in file -- {d:,}')

Total duplicates in file -- 4,728,942


In [24]:
%%time
with lsh.insertion_session() as session:
    with open('d\wiki4_v_01.txt', 'w', encoding='utf-8') as f:
        with open('d\wiki4.txt', encoding='utf-8') as f1:
            for piece in read_in_chunks_lines(f1, chunk_size=6000000):
                sents = [process_txt_new(x) for x in piece]
                sents = [line for line in sents if line is not None]
                hashes = [hashes_simple_1(x) for x in tqdm(sents)]
                
                for i in range(len(hashes)):                   
                    if sents[i] == 'PLACEHOLDER':
                        f.write('\n')  
                    else:    
                        key = hashes[i]
                        q = len(w4.query(key))
                        q1 = len(w0.query(key))
                        q2 = len(w1.query(key))
                        q3 = len(w2.query(key)) 
                        q4 = len(w3.query(key))
                        q5 = len(w5.query(key))  

                        s =  q1 + q2 + q3 + q4 + q5
              
                        if q < 2 and s == 0:
                            f.write(sents[i] + '\n')
                        else:
                            if not len(lsh.query(hashes[i])): 
                                r +=1 
                                session.insert('w'+str(r), hashes[i])                
                                f.write(sents[i] + '\n')
                            else:
                                d += 1
                                if d % 100000 == 0:
                                    print(f'Duplicates found --- {d:,}')

100%|██████████████████████████████████████████████████████████████████████████| 60412/60412 [00:18<00:00, 3302.01it/s]


Duplicates found --- 4,800,000


100%|██████████████████████████████████████████████████████████████████████████| 57366/57366 [00:17<00:00, 3248.74it/s]


Duplicates found --- 4,900,000


100%|██████████████████████████████████████████████████████████████████████████| 53279/53279 [00:17<00:00, 3018.18it/s]


...

Duplicates found --- 5,200,000


100%|██████████████████████████████████████████████████████████████████████████| 55013/55013 [00:18<00:00, 3033.00it/s]


Duplicates found --- 5,300,000


100%|██████████████████████████████████████████████████████████████████████████| 44176/44176 [00:14<00:00, 3023.45it/s]


Wall time: 55min 19s


In [25]:
print(f'Total duplicates in file -- {d:,}')

Total duplicates in file -- 5,372,656


In [26]:
%%time
with lsh.insertion_session() as session:
    with open('d\wiki5_v_01.txt', 'w', encoding='utf-8') as f:
        with open('d\wiki5.txt', encoding='utf-8') as f1:
            for piece in read_in_chunks_lines(f1, chunk_size=5000000):
                sents = [process_txt_new(x) for x in piece]
                sents = [line for line in sents if line is not None]
                hashes = [hashes_simple_1(x) for x in tqdm(sents)]
                
                for i in range(len(hashes)):                   
                    if sents[i] == 'PLACEHOLDER':
                        f.write('\n')  
                    else:    
                        key = hashes[i]
                        q = len(w5.query(key))
                        q1 = len(w0.query(key))
                        q2 = len(w1.query(key))
                        q3 = len(w2.query(key)) 
                        q4 = len(w3.query(key))
                        q5 = len(w4.query(key))  

                        s =  q1 + q2 + q3 + q4 + q5
             
                        if q < 2 and s == 0:
                            f.write(sents[i] + '\n')
                        else:
                            if not len(lsh.query(hashes[i])): 
                                r +=1 
                                session.insert('w'+str(r), hashes[i])                
                                f.write(sents[i] + '\n')
                            else:
                                d += 1
                                if d % 100000 == 0:
                                    print(f'Duplicates found --- {d:,}')

100%|██████████████████████████████████████████████████████████████████████████| 56296/56296 [00:18<00:00, 3053.44it/s]


Duplicates found --- 5,400,000


100%|██████████████████████████████████████████████████████████████████████████| 54117/54117 [00:17<00:00, 3070.95it/s]


Duplicates found --- 5,500,000


100%|██████████████████████████████████████████████████████████████████████████| 55923/55923 [00:18<00:00, 3085.87it/s]


...

Duplicates found --- 6,200,000


100%|██████████████████████████████████████████████████████████████████████████| 57287/57287 [00:18<00:00, 3129.13it/s]


Duplicates found --- 6,300,000


100%|██████████████████████████████████████████████████████████████████████████| 36793/36793 [00:12<00:00, 3052.83it/s]


Wall time: 1h 14min


In [27]:
print(f'Total duplicates in file -- {d:,}')

Total duplicates in file -- 6,386,931


In [28]:
print(f'Number of unique duplicated sentences (hashes) -- {r:,}')

Number of unique duplicated sentences (hashes) -- 1,329,913
